In [ ]:
import os
import sys
import keras_tuner
from dotenv import load_dotenv

from usd_idr_forecasting.utils import get_dt_now
from usd_idr_forecasting.configs import ProjectConfig
from usd_idr_forecasting.tuner import Tuner
from usd_idr_forecasting.models import TemporalHyperModel
from usd_idr_forecasting.data.datasets import DatasetLoader

load_dotenv()
PROJECT_WORKING_DIR = os.getenv("PROJECT_WORKING_DIR")

# **Load Config**

In [ ]:

config_path = os.path.join(PROJECT_WORKING_DIR, 'project_configs.yaml')
project_config = ProjectConfig.from_yaml(config_path)
project_config

# **Load Datasets**

In [ ]:
train_set, valid_set, batch_size, prep_artifact = DatasetLoader(config=project_config).load_dataset_for_training(batch_size=32)

# **Tuning**

In [ ]:
process_id = get_dt_now()
print(f'Process Tuning ID: {process_id}')

In [ ]:

model_lstm_tuner = TemporalHyperModel(config=project_config, model_type='lstm')
lstm_tuner = Tuner(
	hypermodel=model_lstm_tuner,
    objective=keras_tuner.Objective('val_rmse', direction='min'),
    max_trials=2,
    executions_per_trial=1,
    overwrite=False,
    directory="lstm_tuner",
    project_name=f"tuned@lstm_{process_id}",
    config=project_config,
    model_type='lstm',
    process_id=process_id,
)


model_gru_tuner = TemporalHyperModel(config=project_config, model_type='gru')
gru_tuner = Tuner(
	hypermodel=model_gru_tuner,
    objective=keras_tuner.Objective('val_rmse', direction='min'),
    max_trials=2,
    executions_per_trial=1,
    overwrite=False,
    directory="gru_tuner",
    project_name=f"tuned@gru_{process_id}",
    config=project_config,
    model_type='gru',
    process_id=process_id,
)


In [ ]:
lstm_tuner.search_space_summary()

In [ ]:
gru_tuner.search_space_summary()

## **Start Tuning**

### **LSTM**

In [ ]:
lstm_tuner.start(train_set, valid_set)

### **GRU**

In [ ]:
gru_tuner.start(train_set, valid_set)

In [ ]:
print("This will only print to the screen once.")